**Autor:** Felipe Moreira Sallazar 

**Contato:** sallazarfelipe@gmail.com

**Vídeo de explicação:** [Inteligência Artificial - Problema do caixeiro viajante](https://youtu.be/cqa1JKYOi24)


**Problema:**

Suponha  que  um  caixeiro  viajante  tenha  que  visitar  *n*  cidades  diferentes,  iniciando  e encerrando  sua  viagem  na  primeira  cidade.  Suponha,  também,  que  não  importa  a ordem  com  que  as  cidades  são  visitadas  e  que  de  cada  uma  delas  o  caixeiro  pode  ir diretamente para qualquer outra. O problema do caixeiro viajante consiste em descobrir a rota que torna mínima a viagem total. 

**Exercício:**

Implemente  em  Python  um  Algoritmo  Evolutivo  para  a  resolução  do  problema  do Caixeiro viajante considerando 6 (seis) cidades. Em seu código, gere as distâncias entre as cidades de maneira aleatória. Seu programa deve encontrar a melhor rota partindo 
da  cidade  0,  passando  por  todas  as  cidades  e  retornando  à  cidade  0. Defina a melhor maneira de  modelar  seus  indivíduos  que  representam  as  soluções  do  problema,  e  também  a função  de  *fitness*.  Faça  suas  próprias  escolhas  sobre  os  operadores  de  seleção, cruzamento, mutação e elitismo. Não é permitido utilizar bibliotecas prontas.



In [112]:
# Bibliotecas a serem utilizadas
import math
import numpy as np
import random

Declarando os parâmetros do problema.

In [121]:
# Variáveis globais
n = 6                           # Número de cidades
d = np.zeros((n,n), dtype=int)  # Tabela de distância entre as cidades
d_max = 80                      # Distância máxima entre as cidades

# Gerando distâncias aleatórias entre cada cidade
for i in range(n):
  for j in range(i+1, n):
    d[i,j] = d[j,i] = random.randint(1, d_max)

# Imprime Tabela das Distância
print(f"{' ':>{len(str(np.max(d)))}}", " ".join([f"{i:>{len(str(np.max(d)))}}" for i in range(n)]))
for i in range(n):
  print(f"{i:>{len(str(np.max(d)))}}", " ".join([f"{j:>{len(str(np.max(d)))}}" for j in d[i]]))

    0  1  2  3  4  5
 0  0  9 69 72 35 48
 1  9  0 12  3 54 36
 2 69 12  0 76 35 42
 3 72  3 76  0 48 25
 4 35 54 35 48  0 26
 5 48 36 42 25 26  0


Declarando as variáveis para a simulação:

In [122]:
# Criando a população de indivíduos
caminho_max = sum([i for i in range(n)])  # Número máximo de "passos"
num_individuos = 1000  # Número de indivíduo por geração
num_geracoes = 500     # Número de gerações
elite_porcent = 0.01   # Porcentagem de individuos para compor a elite

populacao_inicial = np.zeros((num_individuos, caminho_max), dtype=int)
for i in range(num_individuos):
  for j in range(caminho_max):
    populacao_inicial[i][j] = random.randint(0,n-1)



Definindo as funções para a simulação:

Sendo $d_{i}$ a distância percorrida entre uma cidade e outra pelo trajeto do caxeiro, o problema é minimizar a função $f = \sum d_{i}$, que será nossa função **fit**.

In [108]:
# Funções fitness para o indivíduo e para a população.
def fitness(trajeto:np.array) -> int:
  """Recebe um indivíduo (isto é, um trajeto) e retorna a distância total percorrida, ou seja, a fitness.

  Args:
      trajeto (np.array): indivíduo do tipo lista contendo a ordem das cidades percorridas.
  Returns:
      int: soma total das distância percorrida entre cada cidade (fitness)
  """
  f = 0        # fitnesses
  w = 2*d_max  # Peso da penalidade por não passar em uma cidade 

  # Somando a distância total percorrida
  passos = len(trajeto)
  for idx in range(passos-1):
    cidade1, cidade2 = trajeto[idx], trajeto[idx+1]
    f += d[cidade1, cidade2]

  # Contabilizando as distâncias partindo da e voltando para a cidade 0
  f += d[0, trajeto[0]] + d[trajeto[-1], 0]  

  # Aplicando penalidade para cada cidade que não passou
  f += w*cidades_ignoradas(trajeto)

  return f


def cidades_ignoradas(lista:np.array) -> int:
  """Recebe uma lista e retorna o número de cidades ignoradas.

  Args:
      lista (list): lista de elementos a serem verificados

  Returns:
      int: número de cidades que o indivíduo não passou.
  """
  lista = list(lista)
  cids_ignoradas = []
  for i in range(1, n):  # Ignorando a cidade 0
    if i not in lista and i not in cids_ignoradas: 
      cids_ignoradas.append(i)
  return len(cids_ignoradas)


def retorna_fitnesses(populacao:np.array) -> list:
  """Recebe a matriz contendo a população de indivíduos e chama a função "fitness" para cada um, retornando uma lista com a pontuação de cada.

  Args:
      populacao (np.array): matriz bidimensional do tipo array do numpy com número de colunas igual ao número de cidades no trajeto, e número de linhas igual ao número de indivíduos
  Returns:
      list: retorna a lista de fitness de todos os indivíduos na mesma ordem das linhas da matriz
  """
  fit = []  # Lista dos fitness
  for i in range(len(populacao)):
    f = fitness(populacao[i])
    fit.append(f)
  return fit


# Funções para gerar a próxima geração de indivíduos
def torneio(idx_individuo1:int, idx_individuo2:int, fitnesses:list) -> int:
  """Compara dois indivíduos dentro da lista de fitnesses, retornando o indivíduo com menor valor

  Args:
      idx_individuo1 (int): index do primeiro indivíduo dentro da lista de fitnesses
      idx_individuo2 (int): index do segundo indivíduo dentro da lista de fitnesses
      fitnesses (list): lista dos resultados calculados pela função "retorna_fitnesses"

  Returns:
      int: index do indivíduo de menor valor
  """
  if fitnesses[idx_individuo1] < fitnesses[idx_individuo2]: return idx_individuo1
  else: return idx_individuo2
  


def selecao_torneio(fitnesses:list) -> tuple[int]:
  """Aplica 2 torneios no fitnesses da população, selecionando um dupla de indivíduos em cada torneio, e retorna os dois vencedores dos torneios, sendo estes indivíduos distintos.

  Args:
      fitnesses (list): lista com os fitness dos indivíduos da população.

  Returns:
      tuple[int]: tupla de tamanho 2, com os index dos dois vencedores dos torneios.
  """
  ind1 = -1
  ind2 = -1

  num_individuos = len(fitnesses)

  while ind1 == ind2: 
    # Torneio 1
    sorteados = random.sample(range(0, num_individuos), 2)
    ind1 = torneio(*sorteados, fitnesses)
    # Torneio 2
    sorteados = random.sample(range(0, num_individuos), 2)
    ind2 = torneio(*sorteados, fitnesses)
  return ind1,ind2


def cruzamento(pai1:np.array, pai2:np.array) -> tuple[np.array]:
  """Recebe 2 indivíduos, a partir deles, seleciona um ponto aleatório para fazer a partição de ambos os indivíduos e retorna 2 novos indivíduos (filhos) com os genes cruzados.
 
  Args:
      idx (tuple[int]): Uma tupla de tamanho 2 contendo os index dos indivíduos a serem selecionados dentro da população
      populacao (np.array): _description_

  Returns:
      tuple[np.array]: _description_
  """
  ponto = random.randint(1, len(pai1)-1)

  filho1 = np.concatenate([pai1[:ponto],pai2[ponto:]])
  filho2 = np.concatenate([pai2[:ponto],pai1[ponto:]])

  return filho1, filho2


def elitismo(fitnesses:list, porcentagem:float) -> list:
  """Recebe uma lista com os fitnesses da população e a porcentagem desejada a ser selecionada, retorna uma lista com os index dos indivíduos mais aptos, com a mesma paridade da população total.

  Args:
      fitnesses (list): lista com os fitnesses (int|float) da população.
      porcentagem (float): porcentagem a ser selecionada em relação ao total de indivíduos (na forma decimal, de 0 a 1) 

  Returns:
      list: lista com os index dos indivíduos mais aptos.
  """
  total = len(fitnesses)
  num_elite = int(total*porcentagem)

  # Manter paridade 
  if total%2: num_elite = num_elite + 1 - num_elite%2 
  else: num_elite = num_elite - num_elite%2
  
  # Seleção da elite (nesse formato é possível lidar com fitness repetidos/iguais)
  elite = sorted(range(len(fitnesses)), key=lambda x: fitnesses[x])[:num_elite]
  return elite


def mutacao(filhos:tuple[np.array], taxa1:float, taxa2:float) -> tuple[np.array]:
  """Recebe 2 indivíduos e aplica 2 mutações de acordo com os parâmetros taxa1 e taxa2, retornando os indivíduos com duas, uma ou nenhuma mutação.

  Args:
      filhos (tuple[np.array]): tupla de tamanho 2, contendo os 2 indivíduos
      taxa1 (float): taxa referente a mutação de inverção de ordem de dois genes (na forma decimal, de 0 a 1) 
      taxa2 (float): taxa referente a mutação de transformação aleatória de um gene (na forma decimal, de 0 a 1) 

  Returns:
      tuple[np.array]: tupla de tamanho 2, contendo os 2 indivíduos após aplicado as mutações
  """
  num_pos = len(filhos[0])
  
  for i in range(len(filhos)):
    # Inverter uma cidade com outra qualquer
    if random.random() < taxa1:
      pos1 = random.randint(0, num_pos-1)
      pos2 = random.randint(0, num_pos-1)
      filhos[i][pos1], filhos[i][pos2] = filhos[i][pos2], filhos[i][pos1]

    # Trocar uma cidade por outra qualquer
    if random.random() < taxa2:
      pos = random.randint(0, num_pos-1)
      filhos[i][pos] = random.randint(0,n-1)  

  return filhos

Programa principal para a simulação:

In [123]:
# Simulação
for it in range(num_geracoes+1):

  # Nova populacao vazia
  nova_populacao = np.zeros((num_individuos,caminho_max), dtype=int)

  # Calcula o fitness da população
  fit = retorna_fitnesses(populacao_inicial)
  
  # Imprime o melhor e o pior individuo 
  if it%50 == 0: 
    info_melhor = [fit.index(min(fit)), 0]
    info_melhor[1] = fit[info_melhor[0]]
    info_pior = [fit.index(max(fit)), 0]
    info_pior[1] = fit[info_pior[0]]
    media_pop = np.mean(fit)

    print("Geração:", it, 
          "\nMelhor:",' '.join(map(str,populacao_inicial[info_melhor[0]])), "| Fit:", info_melhor[1], info_melhor[0],
          "\nPior:",' '.join(map(str,populacao_inicial[info_pior[0]])), "| Fit:", info_pior[1], info_pior[0], 
          "\nMédia:", media_pop)
    print()

  # Elitismo
  elite = elitismo(fit, elite_porcent)
  for idx in range(len(elite)): 
    nova_populacao[idx] = populacao_inicial[elite[idx]]

  # Gera os filhos restantes para completar a população
  num_filhos = len(elite)
  while num_filhos < num_individuos:

    # Seleção por torneio
    vancedor1, vancedor2 = selecao_torneio(fit)
    
    # Cruzamento
    filhos = cruzamento(pai1=populacao_inicial[vancedor1], pai2=populacao_inicial[vancedor2]) 

    # Mutacao (50% de chance de cada mutação em cada indivíduo)
    filhos = mutacao(filhos, 0.5, 0.5)

    # Coloca os filhos na nova população
    nova_populacao[num_filhos] = filhos[0]
    nova_populacao[num_filhos+1] = filhos[1]

    # Aumenta o número de filhos
    num_filhos += 2

  # Substitui a populacao antiga pela atual
  populacao_inicial = np.copy(nova_populacao)

  




Geração: 0 
Melhor: 1 1 3 1 3 1 2 2 2 4 5 5 2 5 1 | Fit: 223 117 
Pior: 2 0 2 0 5 0 5 2 5 3 0 3 2 3 4 | Fit: 1068 824 
Média: 582.536

Geração: 50 
Melhor: 1 1 1 1 3 3 3 3 5 5 5 4 2 1 0 | Fit: 119 0 
Pior: 0 1 4 1 1 1 2 2 1 4 4 4 2 1 4 | Fit: 651 358 
Média: 308.607

Geração: 100 
Melhor: 1 1 1 1 3 3 3 3 5 5 5 4 2 1 0 | Fit: 119 0 
Pior: 0 5 1 1 3 1 3 0 5 5 1 1 3 5 1 | Fit: 642 673 
Média: 289.458

Geração: 150 
Melhor: 1 1 1 1 3 3 3 3 5 5 5 4 2 1 0 | Fit: 119 0 
Pior: 0 5 1 1 3 3 0 3 1 5 1 5 5 1 1 | Fit: 707 612 
Média: 280.537

Geração: 200 
Melhor: 1 1 1 1 3 3 3 3 5 5 5 4 2 1 0 | Fit: 119 0 
Pior: 1 1 1 1 4 0 2 0 1 1 2 4 1 1 1 | Fit: 675 277 
Média: 298.634

Geração: 250 
Melhor: 1 1 1 1 3 3 3 3 5 5 5 4 2 1 0 | Fit: 119 0 
Pior: 0 3 1 5 3 1 5 1 1 5 4 5 1 1 5 | Fit: 579 780 
Média: 285.869

Geração: 300 
Melhor: 1 1 1 1 3 3 3 3 5 5 5 4 2 1 0 | Fit: 119 0 
Pior: 1 5 1 2 1 5 1 4 1 5 0 4 2 1 0 | Fit: 620 660 
Média: 282.41

Geração: 350 
Melhor: 1 1 1 1 3 3 3 3 5 5 5 4 2 1 0 | Fit: 119 

Aqui analisamos um caso mais simples do mesmo problema, onde o caixeiro tem que atravessar n cidades sem poder retornar a uma cidade já visitada, desse modo é possível limitar os trajetos realizáveis, podendo assim exaurir o problema ao analisar todas as opções de caminhos possíveis e selecionar a melhor usando os mesmos critérios aplicados ao problema anterior.

In [124]:
# Algoritmo para exaurir as possibilidades para o caso mais simples
def embaralhar(lista:list) -> list:
  lista = lista.copy()
  limite = len(lista)
  for i in range(limite):
    idx = random.randint(0, limite-1)
    lista[i], lista[idx] = lista[idx], lista[i]
  return lista


# Criando a população de indivíduos
num_individuos = math.factorial(n-1)   # Número de possibilidade de indivíduos

populacao = []
cidades = [i for i in range(1, n)]
for i in range(num_individuos):
    while True:
        sample = embaralhar(cidades)
        if sample not in populacao:
           populacao.append(sample)
           break

# Analise do melhor indivíduo
fit = retorna_fitnesses(populacao)
melhor = fit.index(min(fit))
print(populacao[melhor], fitness(populacao[melhor]))

[4, 2, 5, 3, 1] 149
